# Let's try the ChefGPT

Let's try our newly trained model and see how well it performs

In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_5k_v1", device_map="auto")
tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_5k_v1")

In [2]:
#
generation_kwargs_v2 = {
    "max_length": 512,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
#     "temperature": 0.7,
    "top_p": 0.95,
    "num_beams": 5,
    "num_return_sequences": 3,
    "early_stopping": True    
}

#
generation_kwargs_v3 = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3
}

generation_kwargs_v4 = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

def generate_recipe_v1(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to('cuda')
    output_ids = model.generate(input_ids, max_length=512, num_return_sequences=1)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)    
    return output_text

def generate_recipe_v2(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).input_ids
    output_ids = model.generate(input_ids, **generation_kwargs_v2)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def generate_recipe_v3(prompt):
    input_text = f"generate recipe: {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512).input_ids
    output_ids = model.generate(input_ids, **generation_kwargs_v3)
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

def generate_recipe_v4(prompt):
    input_text = f"generate recipe: {prompt}"
    inputs = tokenizer(
        input_text, 
        max_length=512, 
        padding="max_length", 
        truncation=True, 
        return_tensors="pt"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids, 
        attention_mask=attention_mask,
        **generation_kwargs_v4
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text

In [3]:
# food_items = "butter, mushrooms, garlic, zucchini, onion, eggs, Ricotta cheese, shredded Monterey Jack, frozen spinach, dill weed, salt"
# food_items = "chicken breasts, lime marinade, tomatoes, shredded cheese, tortilla chips"
#food_items = "pie shell, eggs, butter, sugar, cocoa, flour, milk, vanilla"
# food_items = "cooking oats, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "cooking oat, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "coconut, sugar, bran cereal, pumpkin seeds, flour, rolled oats, milk"
# food_items = "Salt, Pepper, FILLING, Olive Oil, Sausage, Fresh Spinach Leaves, Eggs, Egg Whites, Coconut Milk, Colby Cheese, Salt, Pepper, Hot Sauce, Green Onions"
food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

recipe = generate_recipe_v1(food_items)
# s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(recipe)
# print(s)

/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


title: Chicken Soup ingredients: 1 lb. chicken, 1 c. peas, 1 c. carrots, 1 c. cream of chicken soup, 1 c. cream of celery soup, 1 c. chicken broth, 1 c. milk, 1 c. Bisquick mix directions: Cook chicken in boiling water. Add peas and carrots. Add soup and chicken broth. Add milk and mix well. Add Bisquick mix. Cook until chicken is tender.


In [5]:
# very CPU intensive!
recipe = generate_recipe_v2(food_items)
# s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(recipe)
# sprint(s)

title: Chicken Casserole ingredients: 1 lb. chicken, 1 c. chopped peas, 2 carrots, chopped, 1 can cream of chicken soup, 2 cans cream of celery soup, 3 cans chicken broth, 1 cup milk, Bisquick mix directions: Preheat oven to 400u00b0F. In a large bowl combine chicken soup milk and milk. Mix well. Pour over chicken. Cover and refrigerate until ready to serve.


In [4]:
recipe = generate_recipe_v3(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
print(s)

/home/datascience/conda/pytorch110_p38_gpu_v1/lib/python3.8/site-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


title: Chicken Pot Pie 

Ingredients: 1 1/2 lb. chicken pot pie mix, 1 large bowl of peas, 1 small can fresh carrots, 1 (1 oz.) can cream of chicken soup, 2 oz. can cream of celery soup, 1 can chicken broth, 1 c. milk, 1/2 c. Bisquick mix 

Directions: Add chicken and peas. Cook for 15 minutes. Add carrots and cream of chicken soup. Add chicken broth and milk. Add Bisquick mix. Bake for 1 hour.


In [5]:
recipe = generate_recipe_v4(food_items)
s = recipe.replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:")
# print(recipe)
print(s)

title: Chicken Soup 

Ingredients: 1 whole chicken breast, 1 c. seasoned peas, 2 cice sliced carrots, 1 can cream of chicken soup, 1 large can cream and celery soup, 2 Tbsp. chicken broth, 2 3/4 cm. milk, 1 (9 1/2 oz.) can Bisquick mix 

Directions: Preheat oven to 375u00b0. In a large bowl combine chicken bones pea pods carrots cream of rice soup and celeries. Cook 4 hours stirring constantly until chicken is cooked through. Add chicken broth. Add milk and mix well. Pour mixture over chicken. Bring to a boil stirring until chicken starts to form foamy. Heat the heated broth and water together. Chill 10 minutes. Cool in pan. Cover and add the meat mixture. Pour the chicken mixture over the chicken. Stir until creamy. Cover with the chicken and cook stirring occasionally until mixture is thickened.


## Using Pipelines

In [6]:
from transformers import pipeline
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_5K_v1")
# tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_5K_v1", model_max_len=512)

# model = T5ForConditionalGeneration.from_pretrained("fine_tuned_t5_recipes_base_10K_v1")
# tokenizer = T5Tokenizer.from_pretrained("fine_tuned_t5_recipes_base_10K_v1", model_max_len=512)

generator = pipeline(model=model, tokenizer=tokenizer, task="text2text-generation")

In [7]:
# food_items = 'cooking oats, brown sugar, salad oil, eggs, salt, almond extract'
# food_items = 'ground beef, vinegar, sugar, chili powder, Worcestershire sauce, catsup, mustard, onion'
# food_items = "cooking oat, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "coconut, sugar, bran cereal, pumpkin seeds, flour, rolled oats, milk"
# food_items = "butter, mushrooms, garlic, zucchini, onion, eggs, Ricotta cheese, shredded Monterey Jack, frozen spinach, dill weed, salt"
# food_items = "cooking oats, brown sugar, salad oil, eggs, salt, almond extract"
# food_items = "Salt, Pepper, FILLING, Olive Oil, Sausage, Fresh Spinach Leaves, Eggs, Egg Whites, Coconut Milk, Colby Cheese, Salt, Pepper, Hot Sauce, Green Onions"
food_items = "chicken, peas, carrots, cream of chicken soup, cream of celery soup, chicken broth, milk, Bisquick mix"

tokenizer_kwargs = {
    "max_length": 512,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95,
    "num_return_sequences": 2
}

response = generator(f"generate recipe: {food_items}", **tokenizer_kwargs)

print(response[0]['generated_text'].replace("ingredients:", "\n\nIngredients:").replace("directions:", "\n\nDirections:"))

title: Chicken Casserole 

Ingredients: 1/2 lb chicken, skin on and bone on, 8 cups peas, cut into small chunks, 8 carrots, skin on and bones removed, 4 quarts cream of chicken soup, 1 large quart cream of celery soup, 4 cups chicken broth, 1 cup milk, 1/2 cup Bisquick mix 

Directions: Boil peas carrots in water until tender then cut into pieces. In a small pot bring chicken broth milk and Bisquick mix to a boil. Reduce heat to medium. Add soup soup and cook for 20 minutes. Add broth and milk. Cover and cook for 20 minutes to 1 hour. Serves 2.
